In [ ]:
import sys
import os
sys.path.append('../')
import pathlib as pl
from SymEigen import *

from sympy import symbols, Matrix, log
from project_dir import backend_source_dir

Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")
Gen.DisableLatexComment()

mu, lmbd = symbols("mu lambda")

X = Eigen.Vector("X", 9)
X0 = X[0:3,0]
X1 = X[3:6,0]
X2 = X[6:9,0]

E01 = X1 - X0
E02 = X2 - X0

A = Matrix(
[
    [E01.dot(E01), E01.dot(E02)],
    [E02.dot(E01), E02.dot(E02)]
])

A = Eigen.FromSympy("A", A)

VecA = A.Vectorize("VecA")

In [ ]:
detA = A.det()

In [ ]:
IB = Eigen.Matrix("IB", 2, 2)

In [ ]:
detIB = IB.det()
detIB

In [ ]:
lnJ = log(detA * detIB)/2
lnJ

In [ ]:
E = (mu / 2 * ((IB * A).trace() - 2 - 2 * lnJ) + lmbd / 2 * lnJ * lnJ)
E

In [ ]:
dEdX = VecDiff(E, X)

In [ ]:
ddEddX = VecDiff(dEdX, X)

In [ ]:
ClA = Gen.Closure(X)
ClE = Gen.Closure(mu, lmbd, X, IB)


In [ ]:
s = f'''
{ClA("A",A)}
{ClE("E",E)}
{ClE("dEdX",dEdX)}
{ClE("ddEddX",ddEddX)}
'''
print(s)

f = open( backend_source_dir('cuda') / 'finite_element/constitutions/sym/shell_neo_hookean_2d.inl', 'w')
f.write(s)
f.close()